In [1]:
from numpy import loadtxt
import tensorflow
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
from category_encoders.target_encoder import TargetEncoder 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
df_train = pd.read_csv('Train_TP2_Datos_2020-2C.csv')
df_test = pd.read_csv('Test_TP2_Datos_2020-2C.csv')

Feature Engineering

In [3]:
train = df_train[(df_train.Stage == 'Closed Won') | (df_train.Stage == 'Closed Lost')]
cambio_binario = {'Closed Lost' : 0, 'Closed Won' : 1}
train['Stage'] = train['Stage'].map(cambio_binario)
train['Opportunity_Created_Date'] = pd.to_datetime(train['Opportunity_Created_Date'])
train['años'] = train['Opportunity_Created_Date'].dt.year
train['meses'] = train['Opportunity_Created_Date'].dt.month
train.sort_values(['años', 'meses'], ascending=True, inplace=True)
train

<ipython-input-3-edd5f32b7b21>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Stage'] = train['Stage'].map(cambio_binario)
<ipython-input-3-edd5f32b7b21>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Opportunity_Created_Date'] = pd.to_datetime(train['Opportunity_Created_Date'])
<ipython-input-3-edd5f32b7b21>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,años,meses
16104,12714,EMEA,France,0,0,0,0,0,Bureaucratic_Code_4,7/27/2013,...,NaT,2,EUR,662287.50,EUR,662287.50,0,Prod_Category_A_None,2013,11
16105,18684,APAC,Philippines,1,1,1,1,0,Bureaucratic_Code_3,1/22/2014,...,NaT,20,USD,9803430.00,USD,9803430.00,0,Prod_Category_A_None,2014,3
16106,20675,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,8/22/2013,...,NaT,0,USD,170931.60,USD,341863.20,0,Prod_Category_A_None,2014,3
16107,20682,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,8/22/2013,...,NaT,0,USD,170931.60,USD,341863.20,0,Prod_Category_A_None,2014,3
16108,20513,Americas,None,1,0,1,0,0,Bureaucratic_Code_5,8/22/2013,...,NaT,6,USD,4395304.00,USD,31410780.00,1,Prod_Category_A_None,2014,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16099,22652,Japan,Japan,1,1,0,0,0,Bureaucratic_Code_4,9/23/2013,...,NaT,0,JPY,1071840.00,JPY,1071840.00,1,Prod_Category_A_None,2018,12
16100,20803,Americas,NW America,1,1,1,1,0,Bureaucratic_Code_4,7/27/2013,...,NaT,1,USD,309441.60,USD,309441.60,1,Prod_Category_A_None,2018,12
16101,5839,Americas,SE America,0,0,0,0,0,Bureaucratic_Code_4,6/15/2016,...,NaT,0,USD,1198.53,USD,22157.43,1,Prod_Category_A_None,2018,12
16102,5845,Americas,SE America,0,0,0,0,0,Bureaucratic_Code_4,6/15/2016,...,NaT,0,USD,5752.50,USD,22157.43,1,Prod_Category_A_None,2018,12


In [4]:
cambio_binario = {'AUD' : 0.77, 'EUR' : 1.21, 'GBP':1.40, 'JPY':0.0094, 'USD':1}
train['moneda'] = train['Total_Taxable_Amount_Currency']
train['moneda'] = train['moneda'].map(cambio_binario)
train

<ipython-input-4-4063ca373737>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['moneda'] = train['Total_Taxable_Amount_Currency']
<ipython-input-4-4063ca373737>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['moneda'] = train['moneda'].map(cambio_binario)


,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,años,meses,moneda
16104,12714,EMEA,France,0,0,0,0,0,Bureaucratic_Code_4,7/27/2013,...,2,EUR,662287.50,EUR,662287.50,0,Prod_Category_A_None,2013,11,1.2100
16105,18684,APAC,Philippines,1,1,1,1,0,Bureaucratic_Code_3,1/22/2014,...,20,USD,9803430.00,USD,9803430.00,0,Prod_Category_A_None,2014,3,1.0000
16106,20675,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,8/22/2013,...,0,USD,170931.60,USD,341863.20,0,Prod_Category_A_None,2014,3,1.0000
16107,20682,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,8/22/2013,...,0,USD,170931.60,USD,341863.20,0,Prod_Category_A_None,2014,3,1.0000
16108,20513,Americas,None,1,0,1,0,0,Bureaucratic_Code_5,8/22/2013,...,6,USD,4395304.00,USD,31410780.00,1,Prod_Category_A_None,2014,5,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16099,22652,Japan,Japan,1,1,0,0,0,Bureaucratic_Code_4,9/23/2013,...,0,JPY,1071840.00,JPY,1071840.00,1,Prod_Category_A_None,2018,12,0.0094
16100,20803,Americas,NW America,1,1,1,1,0,Bureaucratic_Code_4,7/27/2013,...,1,USD,309441.60,USD,309441.60,1,Prod_Category_A_None,2018,12,1.0000
16101,5839,Americas,SE America,0,0,0,0,0,Bureaucratic_Code_4,6/15/2016,...,0,USD,1198.53,USD,22157.43,1,Prod_Category_A_None,2018,12,1.0000
16102,5845,Americas,SE America,0,0,0,0,0,Bureaucratic_Code_4,6/15/2016,...,0,USD,5752.50,USD,22157.43,1,Prod_Category_A_None,2018,12,1.0000


In [5]:
train['Amount_converted'] = train['moneda']*train['Total_Taxable_Amount']
train

<ipython-input-5-22d61439ce36>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Amount_converted'] = train['moneda']*train['Total_Taxable_Amount']


,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,años,meses,moneda,Amount_converted
16104,12714,EMEA,France,0,0,0,0,0,Bureaucratic_Code_4,7/27/2013,...,EUR,662287.50,EUR,662287.50,0,Prod_Category_A_None,2013,11,1.2100,8.013679e+05
16105,18684,APAC,Philippines,1,1,1,1,0,Bureaucratic_Code_3,1/22/2014,...,USD,9803430.00,USD,9803430.00,0,Prod_Category_A_None,2014,3,1.0000,9.803430e+06
16106,20675,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,8/22/2013,...,USD,170931.60,USD,341863.20,0,Prod_Category_A_None,2014,3,1.0000,3.418632e+05
16107,20682,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,8/22/2013,...,USD,170931.60,USD,341863.20,0,Prod_Category_A_None,2014,3,1.0000,3.418632e+05
16108,20513,Americas,None,1,0,1,0,0,Bureaucratic_Code_5,8/22/2013,...,USD,4395304.00,USD,31410780.00,1,Prod_Category_A_None,2014,5,1.0000,3.141078e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16099,22652,Japan,Japan,1,1,0,0,0,Bureaucratic_Code_4,9/23/2013,...,JPY,1071840.00,JPY,1071840.00,1,Prod_Category_A_None,2018,12,0.0094,1.007530e+04
16100,20803,Americas,NW America,1,1,1,1,0,Bureaucratic_Code_4,7/27/2013,...,USD,309441.60,USD,309441.60,1,Prod_Category_A_None,2018,12,1.0000,3.094416e+05
16101,5839,Americas,SE America,0,0,0,0,0,Bureaucratic_Code_4,6/15/2016,...,USD,1198.53,USD,22157.43,1,Prod_Category_A_None,2018,12,1.0000,2.215743e+04
16102,5845,Americas,SE America,0,0,0,0,0,Bureaucratic_Code_4,6/15/2016,...,USD,5752.50,USD,22157.43,1,Prod_Category_A_None,2018,12,1.0000,2.215743e+04


In [6]:
train['Last_Modified_Date'] = pd.to_datetime(train['Last_Modified_Date'])
train['tiempo_op_abierta']=train['Last_Modified_Date']-train['Opportunity_Created_Date']
train['tiempo_op_abierta']= train['tiempo_op_abierta'].dt.days

<ipython-input-6-ad42b1452f89>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Last_Modified_Date'] = pd.to_datetime(train['Last_Modified_Date'])
<ipython-input-6-ad42b1452f89>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['tiempo_op_abierta']=train['Last_Modified_Date']-train['Opportunity_Created_Date']
<ipython-input-6-ad42b1452f89>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [7]:
df_ML_train = train[['años', 'meses', 'TRF', 'Region', 'Billing_Country', 'Opportunity_Owner', 'Opportunity_Type', 'Stage', 'Amount_converted','Product_Family', 'Product_Name','tiempo_op_abierta']]
df_ML_train

,años,meses,TRF,Region,Billing_Country,Opportunity_Owner,Opportunity_Type,Stage,Amount_converted,Product_Family,Product_Name,tiempo_op_abierta
16104,2013,11,2,EMEA,France,Person_Name_16,Opportunity_Type_1,0,8.013679e+05,Product_Family_133,Product_Name_221,1775
16105,2014,3,20,APAC,Germany,Person_Name_4,Opportunity_Type_19,0,9.803430e+06,Product_Family_77,Product_Name_98,859
16106,2014,3,0,Americas,United States,Person_Name_8,Opportunity_Type_1,0,3.418632e+05,Product_Family_164,Product_Name_310,908
16107,2014,3,0,Americas,United States,Person_Name_8,Opportunity_Type_1,0,3.418632e+05,Product_Family_164,Product_Name_310,908
16108,2014,5,6,Americas,United States,Person_Name_12,Opportunity_Type_19,1,3.141078e+07,Product_Family_158,Product_Name_287,485
...,...,...,...,...,...,...,...,...,...,...,...,...
16099,2018,12,0,Japan,Japan,Person_Name_32,Opportunity_Type_1,1,1.007530e+04,Product_Family_132,Product_Name_231,1
16100,2018,12,1,Americas,United States,Person_Name_8,Opportunity_Type_7,1,3.094416e+05,Product_Family_133,Product_Name_220,0
16101,2018,12,0,Americas,United States,Person_Name_38,Opportunity_Type_7,1,2.215743e+04,Product_Family_240,Product_Name_505,0
16102,2018,12,0,Americas,United States,Person_Name_38,Opportunity_Type_7,1,2.215743e+04,Product_Family_140,Product_Name_241,0


# ENCODEAR

In [8]:
cat_cols = df_ML_train.select_dtypes(include='object').columns
cat_cols

Index(['Region', 'Billing_Country', 'Opportunity_Owner', 'Opportunity_Type',
       'Product_Family', 'Product_Name'],
      dtype='object')

In [9]:
df_ML_cat = df_ML_train[cat_cols]
df_ML_cat

,Region,Billing_Country,Opportunity_Owner,Opportunity_Type,Product_Family,Product_Name
16104,EMEA,France,Person_Name_16,Opportunity_Type_1,Product_Family_133,Product_Name_221
16105,APAC,Germany,Person_Name_4,Opportunity_Type_19,Product_Family_77,Product_Name_98
16106,Americas,United States,Person_Name_8,Opportunity_Type_1,Product_Family_164,Product_Name_310
16107,Americas,United States,Person_Name_8,Opportunity_Type_1,Product_Family_164,Product_Name_310
16108,Americas,United States,Person_Name_12,Opportunity_Type_19,Product_Family_158,Product_Name_287
...,...,...,...,...,...,...
16099,Japan,Japan,Person_Name_32,Opportunity_Type_1,Product_Family_132,Product_Name_231
16100,Americas,United States,Person_Name_8,Opportunity_Type_7,Product_Family_133,Product_Name_220
16101,Americas,United States,Person_Name_38,Opportunity_Type_7,Product_Family_240,Product_Name_505
16102,Americas,United States,Person_Name_38,Opportunity_Type_7,Product_Family_140,Product_Name_241


In [10]:
enc = TargetEncoder(cols=df_ML_cat, handle_unknown='value', return_df=False)
enc.fit(df_ML_cat,df_ML_train['Stage'])
df_mean_encoding = pd.DataFrame(enc.transform(df_ML_cat))

/home/tomas/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [11]:
df_mean_encoding.rename(columns={0:'Region', 1:'Billing_Country', 2:'Opportunity_Owner', 3:'Opportunity_Type',4:'Product_Family',5:'Product_Name'}, inplace=True)
df_mean_encoding

,Region,Billing_Country,Opportunity_Owner,Opportunity_Type,Product_Family,Product_Name
0,0.530258,0.280303,0.286996,0.533953,0.735789,0.574468
1,0.583205,0.567527,0.471533,0.211223,0.488476,0.090909
2,0.455886,0.463061,0.587201,0.533953,0.291169,0.551724
3,0.455886,0.463061,0.587201,0.533953,0.291169,0.551724
4,0.455886,0.463061,0.304348,0.211223,0.303922,0.269231
...,...,...,...,...,...,...
16878,0.682481,0.684308,0.783942,0.533953,0.636842,0.748700
16879,0.455886,0.463061,0.587201,0.684201,0.735789,0.659306
16880,0.455886,0.463061,0.348000,0.684201,0.469388,0.469388
16881,0.455886,0.463061,0.348000,0.684201,0.680519,0.752351


In [12]:
df_ML_train = pd.concat([df_ML_train.drop(columns=cat_cols), df_mean_encoding], axis=1)
df_ML_train

,años,meses,TRF,Stage,Amount_converted,tiempo_op_abierta,Region,Billing_Country,Opportunity_Owner,Opportunity_Type,Product_Family,Product_Name
0,2015.0,12.0,10.0,0.0,6380088.00,189.0,0.530258,0.280303,0.286996,0.533953,0.735789,0.574468
1,2015.0,12.0,0.0,1.0,58358.30,39.0,0.583205,0.567527,0.471533,0.211223,0.488476,0.090909
2,2015.0,12.0,0.0,1.0,83865.60,296.0,0.455886,0.463061,0.587201,0.533953,0.291169,0.551724
3,2015.0,12.0,14.0,0.0,7421881.50,840.0,0.455886,0.463061,0.587201,0.533953,0.291169,0.551724
4,2015.0,12.0,25.0,0.0,13357192.50,840.0,0.455886,0.463061,0.304348,0.211223,0.303922,0.269231
...,...,...,...,...,...,...,...,...,...,...,...,...
16942,2015.0,12.0,0.0,1.0,362655.15,46.0,NaN,NaN,NaN,NaN,NaN,NaN
16943,2015.0,12.0,0.0,1.0,362655.15,46.0,NaN,NaN,NaN,NaN,NaN,NaN
16944,2015.0,12.0,0.0,1.0,362655.15,46.0,NaN,NaN,NaN,NaN,NaN,NaN
16945,2015.0,12.0,4.0,0.0,0.00,48.0,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_ML_train = df_ML_train.dropna()
df_ML_train

,años,meses,TRF,Stage,Amount_converted,tiempo_op_abierta,Region,Billing_Country,Opportunity_Owner,Opportunity_Type,Product_Family,Product_Name
0,2015.0,12.0,10.0,0.0,6.380088e+06,189.0,0.530258,0.280303,0.286996,0.533953,0.735789,0.574468
1,2015.0,12.0,0.0,1.0,5.835830e+04,39.0,0.583205,0.567527,0.471533,0.211223,0.488476,0.090909
2,2015.0,12.0,0.0,1.0,8.386560e+04,296.0,0.455886,0.463061,0.587201,0.533953,0.291169,0.551724
3,2015.0,12.0,14.0,0.0,7.421882e+06,840.0,0.455886,0.463061,0.587201,0.533953,0.291169,0.551724
4,2015.0,12.0,25.0,0.0,1.335719e+07,840.0,0.455886,0.463061,0.304348,0.211223,0.303922,0.269231
...,...,...,...,...,...,...,...,...,...,...,...,...
16878,2015.0,12.0,0.0,0.0,5.066976e+03,126.0,0.682481,0.684308,0.783942,0.533953,0.636842,0.748700
16879,2015.0,12.0,0.0,0.0,5.066976e+03,126.0,0.455886,0.463061,0.587201,0.684201,0.735789,0.659306
16880,2015.0,12.0,0.0,0.0,4.612392e+03,647.0,0.455886,0.463061,0.348000,0.684201,0.469388,0.469388
16881,2015.0,12.0,0.0,0.0,4.612392e+03,647.0,0.455886,0.463061,0.348000,0.684201,0.680519,0.752351


Separacion para testeo

In [14]:
df_ML_train.sort_values(['años', 'meses'], ascending=True, inplace=True)

<ipython-input-14-b031bf0c32fa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ML_train.sort_values(['años', 'meses'], ascending=True, inplace=True)


In [15]:
lista_features = df_ML_train.columns.tolist()
lista_features.remove('Stage')
cantidad_features = len(lista_features)
cantidad_features

11

In [16]:
features = df_ML_train[lista_features].values
labels = df_ML_train['Stage'].values

In [17]:
min_max_scaler = preprocessing.MinMaxScaler()
features_scale = min_max_scaler.fit_transform(features)

In [18]:
feature_train, feature_val_and_test, label_train, label_val_and_test = train_test_split(features_scale, labels, test_size=None)
feature_val, feature_test, label_val, label_test = train_test_split(feature_val_and_test, label_val_and_test, test_size=None)

Modelo

In [19]:
model = Sequential()
model.add(Dense(8, input_dim=cantidad_features, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [20]:
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [21]:
model.fit(feature_train, label_train, epochs=100, batch_size=10,verbose = 0)

In [22]:
_, accuracy = model.evaluate(feature_val, label_val)
print('Accuracy: %.2f' % (accuracy*100))

99/99 [==============================] - 0s 2ms/step - loss: 0.1736 - accuracy: 0.7412
Accuracy: 74.12


In [23]:
prototipo = model.predict(feature_test)
prototipo

array([[0.01582751],
       [0.7712494 ],
       [0.6553167 ],
       ...,
       [0.789165  ],
       [0.34554112],
       [0.23795196]], dtype=float32)

In [66]:
df_prototipo = pd.DataFrame(prototipo, columns = ['a'])
df_prototipo.a.value_counts()

0.583150    11
0.764139     2
0.822851     2
0.894897     2
0.358725     1
            ..
0.684865     1
0.201489     1
0.596972     1
0.047138     1
0.806644     1
Name: a, Length: 1039, dtype: int64

Test

In [25]:
df_test

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Month,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A
0,6140,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,2019 - 5,Q2,2019,NaT,0,EUR,162240.0,EUR,367419.0,Prod_Category_A_None
1,6146,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,2019 - 5,Q2,2019,NaT,0,EUR,78624.0,EUR,367419.0,Prod_Category_A_None
2,6151,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,2019 - 5,Q2,2019,NaT,0,EUR,126555.0,EUR,367419.0,Prod_Category_A_None
3,6118,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,2019 - 6,Q2,2019,NaT,1,EUR,243360.0,EUR,757783.5,Prod_Category_A_None
4,6124,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,...,2019 - 6,Q2,2019,NaT,0,EUR,157248.0,EUR,757783.5,Prod_Category_A_None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2546,16345,EMEA,KSA,1,1,0,0,0,Bureaucratic_Code_4,6/12/2017,...,2019 - 5,Q2,2019,NaT,0,USD,124740.0,USD,147750.0,Prod_Category_A_None
2547,15218,Americas,SE America,1,1,0,0,0,Bureaucratic_Code_4,6/8/2018,...,2019 - 10,Q4,2019,NaT,0,USD,45054.9,USD,45054.9,Prod_Category_A_None
2548,15224,Americas,SE America,1,1,1,1,0,Bureaucratic_Code_4,6/8/2018,...,2019 - 10,Q4,2019,NaT,0,USD,100122.0,USD,100122.0,Prod_Category_A_None
2549,7286,Americas,NE America,1,1,0,0,0,Bureaucratic_Code_4,8/29/2018,...,2019 - 8,Q3,2019,NaT,0,USD,143220.0,USD,143220.0,Prod_Category_A_None


In [26]:
df_test['Opportunity_Created_Date'] = pd.to_datetime(df_test['Opportunity_Created_Date'])
df_test['años'] = df_test['Opportunity_Created_Date'].dt.year
df_test['meses'] = df_test['Opportunity_Created_Date'].dt.month
df_test.sort_values(['años', 'meses'], ascending=True, inplace=True)
df_test

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A,años,meses
116,17805,Americas,NW America,0,0,0,0,0,Bureaucratic_Code_4,7/27/2013,...,2019,NaT,0,USD,14414.40,USD,14414.40,Prod_Category_A_None,2019,1
117,20075,Americas,NE America,0,0,0,0,0,Bureaucratic_Code_4,8/22/2013,...,2019,NaT,0,USD,4851.00,USD,4851.00,Prod_Category_A_None,2019,1
118,7236,Americas,NE America,1,0,1,0,0,Bureaucratic_Code_5,8/29/2018,...,2019,NaT,0,USD,4504.50,USD,36179.55,Prod_Category_A_None,2019,1
119,7241,Americas,NE America,1,0,1,0,0,Bureaucratic_Code_5,8/29/2018,...,2019,NaT,0,USD,4885.65,USD,36179.55,Prod_Category_A_None,2019,1
120,7247,Americas,NE America,1,0,1,0,0,Bureaucratic_Code_5,8/29/2018,...,2019,NaT,0,USD,4857.60,USD,36179.55,Prod_Category_A_None,2019,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2546,16345,EMEA,KSA,1,1,0,0,0,Bureaucratic_Code_4,6/12/2017,...,2019,NaT,0,USD,124740.00,USD,147750.00,Prod_Category_A_None,2019,4
2547,15218,Americas,SE America,1,1,0,0,0,Bureaucratic_Code_4,6/8/2018,...,2019,NaT,0,USD,45054.90,USD,45054.90,Prod_Category_A_None,2019,4
2548,15224,Americas,SE America,1,1,1,1,0,Bureaucratic_Code_4,6/8/2018,...,2019,NaT,0,USD,100122.00,USD,100122.00,Prod_Category_A_None,2019,4
2549,7286,Americas,NE America,1,1,0,0,0,Bureaucratic_Code_4,8/29/2018,...,2019,NaT,0,USD,143220.00,USD,143220.00,Prod_Category_A_None,2019,4


In [27]:
cambio_binario = {'AUD' : 0.77, 'EUR' : 1.21, 'GBP':1.40, 'JPY':0.0094, 'USD':1}
df_test['moneda'] = df_test['Total_Taxable_Amount_Currency']
df_test['moneda'] = df_test['moneda'].map(cambio_binario)
df_test['Amount_converted'] = df_test['moneda']*df_test['Total_Taxable_Amount']

df_test['Last_Modified_Date'] = pd.to_datetime(df_test['Last_Modified_Date'])
df_test['tiempo_op_abierta']=df_test['Last_Modified_Date']-df_test['Opportunity_Created_Date']
df_test['tiempo_op_abierta']= df_test['tiempo_op_abierta'].dt.days

In [28]:
df_ML_test = df_test[['años', 'meses', 'TRF', 'Region', 'Billing_Country', 'Opportunity_Owner', 'Opportunity_Type', 'Amount_converted','Product_Family', 'Product_Name','tiempo_op_abierta', 'Opportunity_ID']]
df_ML_test

,años,meses,TRF,Region,Billing_Country,Opportunity_Owner,Opportunity_Type,Amount_converted,Product_Family,Product_Name,tiempo_op_abierta,Opportunity_ID
116,2019,1,0,Americas,United States,Person_Name_64,Opportunity_Type_4,14414.400,Product_Family_168,Product_Name_322,1,10756
117,2019,1,0,Americas,United States,Person_Name_9,Opportunity_Type_1,4851.000,Product_Family_213,Product_Name_447,74,10757
118,2019,1,0,Americas,United States,Person_Name_9,Opportunity_Type_8,36179.550,Product_Family_213,Product_Name_447,49,10758
119,2019,1,0,Americas,United States,Person_Name_9,Opportunity_Type_8,36179.550,Product_Family_162,Product_Name_298,49,10758
120,2019,1,0,Americas,United States,Person_Name_9,Opportunity_Type_8,36179.550,Product_Family_168,Product_Name_322,49,10758
...,...,...,...,...,...,...,...,...,...,...,...,...
2546,2019,4,0,EMEA,South Africa,Person_Name_13,Opportunity_Type_1,147750.000,Product_Family_213,Product_Name_447,0,12364
2547,2019,4,0,Americas,United States,Person_Name_38,Opportunity_Type_1,45054.900,Product_Family_221,Product_Name_477,1,12365
2548,2019,4,0,Americas,United States,Person_Name_38,Opportunity_Type_1,100122.000,Product_Family_221,Product_Name_477,1,12366
2549,2019,4,0,Americas,United States,Person_Name_9,Opportunity_Type_7,143220.000,Product_Family_156,Product_Name_281,0,12367


In [29]:
df_ML_test_cat = df_ML_test[cat_cols]
df_ML_test_cat

,Region,Billing_Country,Opportunity_Owner,Opportunity_Type,Product_Family,Product_Name
116,Americas,United States,Person_Name_64,Opportunity_Type_4,Product_Family_168,Product_Name_322
117,Americas,United States,Person_Name_9,Opportunity_Type_1,Product_Family_213,Product_Name_447
118,Americas,United States,Person_Name_9,Opportunity_Type_8,Product_Family_213,Product_Name_447
119,Americas,United States,Person_Name_9,Opportunity_Type_8,Product_Family_162,Product_Name_298
120,Americas,United States,Person_Name_9,Opportunity_Type_8,Product_Family_168,Product_Name_322
...,...,...,...,...,...,...
2546,EMEA,South Africa,Person_Name_13,Opportunity_Type_1,Product_Family_213,Product_Name_447
2547,Americas,United States,Person_Name_38,Opportunity_Type_1,Product_Family_221,Product_Name_477
2548,Americas,United States,Person_Name_38,Opportunity_Type_1,Product_Family_221,Product_Name_477
2549,Americas,United States,Person_Name_9,Opportunity_Type_7,Product_Family_156,Product_Name_281


In [30]:
enc = TargetEncoder(cols=df_ML_test_cat, handle_unknown='value', return_df=False)
enc.fit(df_ML_test_cat,df_ML_test['Opportunity_ID'])
df_mean_encoding_test = pd.DataFrame(enc.transform(df_ML_test_cat))
df_mean_encoding_test

/home/tomas/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,0,1,2,3,4,5
0,11583.837598,11579.642285,11625.741176,11610.415657,11565.035806,11598.571429
1,11583.837598,11579.642285,11707.037037,11504.536007,11561.680272,11509.916667
2,11583.837598,11579.642285,11707.037037,11593.290076,11561.680272,11509.916667
3,11583.837598,11579.642285,11707.037037,11593.290076,11528.062500,11546.858333
4,11583.837598,11579.642285,11707.037037,11593.290076,11565.035806,11598.571429
...,...,...,...,...,...,...
2546,11503.219570,12145.074303,11531.962733,11504.536007,11561.680272,11509.916667
2547,11583.837598,11579.642285,11552.820513,11504.536007,11452.991870,11716.344828
2548,11583.837598,11579.642285,11552.820513,11504.536007,11452.991870,11716.344828
2549,11583.837598,11579.642285,11707.037037,11647.177469,11732.142857,11763.892857


In [31]:
df_mean_encoding_test.rename(columns={0:'Region', 1:'Billing_Country', 2:'Opportunity_Owner', 3:'Opportunity_Type',4:'Product_Family',5:'Product_Name'}, inplace=True)
df_mean_encoding_test

,Region,Billing_Country,Opportunity_Owner,Opportunity_Type,Product_Family,Product_Name
0,11583.837598,11579.642285,11625.741176,11610.415657,11565.035806,11598.571429
1,11583.837598,11579.642285,11707.037037,11504.536007,11561.680272,11509.916667
2,11583.837598,11579.642285,11707.037037,11593.290076,11561.680272,11509.916667
3,11583.837598,11579.642285,11707.037037,11593.290076,11528.062500,11546.858333
4,11583.837598,11579.642285,11707.037037,11593.290076,11565.035806,11598.571429
...,...,...,...,...,...,...
2546,11503.219570,12145.074303,11531.962733,11504.536007,11561.680272,11509.916667
2547,11583.837598,11579.642285,11552.820513,11504.536007,11452.991870,11716.344828
2548,11583.837598,11579.642285,11552.820513,11504.536007,11452.991870,11716.344828
2549,11583.837598,11579.642285,11707.037037,11647.177469,11732.142857,11763.892857


In [32]:
df_ML_test = pd.concat([df_ML_test.drop(columns=cat_cols), df_mean_encoding_test], axis=1)
df_ML_test

,años,meses,TRF,Amount_converted,tiempo_op_abierta,Opportunity_ID,Region,Billing_Country,Opportunity_Owner,Opportunity_Type,Product_Family,Product_Name
0,2019,4,0,444576.990,0,10689,11583.837598,11579.642285,11625.741176,11610.415657,11565.035806,11598.571429
1,2019,4,0,444576.990,0,10689,11583.837598,11579.642285,11707.037037,11504.536007,11561.680272,11509.916667
2,2019,4,0,444576.990,0,10689,11583.837598,11579.642285,11707.037037,11593.290076,11561.680272,11509.916667
3,2019,4,1,916918.035,13,10690,11583.837598,11579.642285,11707.037037,11593.290076,11528.062500,11546.858333
4,2019,4,0,916918.035,13,10690,11583.837598,11579.642285,11707.037037,11593.290076,11565.035806,11598.571429
...,...,...,...,...,...,...,...,...,...,...,...,...
2546,2019,4,0,147750.000,0,12364,11503.219570,12145.074303,11531.962733,11504.536007,11561.680272,11509.916667
2547,2019,4,0,45054.900,1,12365,11583.837598,11579.642285,11552.820513,11504.536007,11452.991870,11716.344828
2548,2019,4,0,100122.000,1,12366,11583.837598,11579.642285,11552.820513,11504.536007,11452.991870,11716.344828
2549,2019,4,0,143220.000,0,12367,11583.837598,11579.642285,11707.037037,11647.177469,11732.142857,11763.892857


In [33]:
test_lista_features = df_ML_test.columns.tolist()
test_lista_features.remove('Opportunity_ID')
test_features = df_ML_test[test_lista_features].values

In [60]:
min_max_scaler = preprocessing.MinMaxScaler()
test_features_scale = min_max_scaler.fit_transform(test_features)

In [61]:
pred = model.predict_proba(test_features)
pred

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [62]:
pred2 = pd.DataFrame(pred, columns=['a'])
pred2.a.value_counts()

1.00000    2539
0.58315      10
0.00000       2
Name: a, dtype: int64

In [47]:
predicciones = pd.Series(pred[:,0])
resultados = pd.concat([df_ML_test['Opportunity_ID'], predicciones], axis=1).rename(columns={0: 'Target'})
resultados.head(10)

,Opportunity_ID,Target
0,10689,1.0
1,10689,1.0
2,10689,1.0
3,10690,1.0
4,10690,1.0
5,10690,1.0
6,10690,1.0
7,10690,1.0
8,10691,1.0
9,10692,1.0


In [48]:
res_promedio = resultados.groupby('Opportunity_ID').mean()

In [49]:
res_promedio.reset_index(inplace=True)
res_promedio

,Opportunity_ID,Target
0,10689,1.0
1,10690,1.0
2,10691,1.0
3,10692,1.0
4,10693,1.0
...,...,...
1562,12364,1.0
1563,12365,1.0
1564,12366,1.0
1565,12367,1.0


In [50]:
res_promedio.Target.unique()

array([1.        , 0.        , 0.58315015, 0.7221001 ], dtype=float32)

In [51]:
((res_promedio['Opportunity_ID'].value_counts()) > 1).value_counts()

False    1567
Name: Opportunity_ID, dtype: int64

In [42]:
res_promedio.drop_duplicates(subset='Opportunity_ID', inplace=True)
res_promedio

,Opportunity_ID,Target
0,10689,1.0
1,10690,1.0
2,10691,1.0
3,10692,1.0
4,10693,1.0
...,...,...
1562,12364,1.0
1563,12365,1.0
1564,12366,1.0
1565,12367,1.0


In [43]:
res_promedio.to_csv('resultsNeuronalNetwork.csv', index=False)